In [ ]:
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint 
import nltk
from nltk.corpus import stopwords
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from keras.models import Model
import chardet
from keras.layers import Input, Dense, Flatten, Dropout, Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVC
from scipy import sparse, stats
from nltk import tokenize
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import metrics
import tensorflow as tf
from keras.layers import Input, Dense, Embedding,Dropout,Conv1D,MaxPooling1D,Flatten,concatenate
def split_text(filepath, min_char):
    file = open(filepath, "r", encoding="utf8")
    text = file.read().replace('\n', ' ')
    text = text.replace('--', ' ').replace('. . .', '').replace('_', '')
    text = text.replace('.”', '”.').replace('."', '".').replace('?”', '”?').replace('!”', '”!')
    file.close()
    sentences = tokenize.sent_tokenize(text)
    sentences = [sent for sent in sentences if len(sent) >= min_char]
    return list(sentences)
min_char = 5
alcott = split_text('Books/Little_Women.txt', min_char = min_char)
stoker = split_text('Books/Dracula.txt', min_char = min_char)
twain = split_text('Books/Tom_Sawyer.txt', min_char = min_char)\
        + split_text('Books/Huckleberry_Finn.txt', min_char = min_char)
austen = split_text('Books/Pride_and_Prejudice.txt', min_char = min_char)\
         + split_text('Books/Emma.txt', min_char = min_char)
collins = split_text('Books/Woman_in_White.txt', min_char = min_char)
doyle = split_text('Books/Study_in_Scarlet.txt', min_char = min_char)\
        + split_text('Books/Sign_of_the_Four.txt', min_char = min_char)\
        + split_text('Books/Hound_of_the_Baskervilles.txt', min_char = min_char)
montgomery = split_text('Books/Anne_of_Green_Gables.txt', min_char = min_char)\
             + split_text('Books/Anne_of_Avonlea.txt', min_char = min_char)
bronte = split_text('Books/Jane_Eyre.txt', min_char = min_char)

text_dict = {'Alcott': alcott, 'Stoker': stoker, 'Twain': twain, 'Austen': austen, 'Collins': collins,
             'Doyle': doyle, 'Montgomery': montgomery, 'Bronte': bronte }
np.random.seed(1)
combined = []
max_length = 8500
names = [alcott, stoker,twain, austen, collins, doyle, montgomery, bronte]
for name in names:
    name = np.random.choice(name, max_length, replace = False)
    combined += list(name)

print('Length of the combined list is ', len(combined))

In [ ]:
labels = ['Alcott']*max_length + ['Stoker']*max_length + ['Twain']*max_length + ['Austen']*max_length\
         + ['Collins']*max_length + ['Doyle']*max_length + ['Montgomery']*max_length + ['Bronte']*max_length
random.seed(3)
zipped = list(zip(combined, labels))
random.shuffle(zipped)
combined, labels = zip(*zipped)
out_data = pd.DataFrame()
out_data['text'] = combined
out_data['author'] = labels
out_data.to_csv('author_data.csv', index=False)
with open("C:/Users/ashwi/author_attribution-master/author_data.csv", 'rb') as file:
    print(chardet.detect(file.read()))
data = pd.read_csv("C:/Users/ashwi/author_attribution-master/author_data.csv", encoding="utf-8")

In [ ]:
# Create word count and character count lists
text = list(data['text'].values)
author = list(data['author'].values)
word_count = []
char_count = []

for i in range(len(text)):
    word_count.append(len(text[i].split()))
    char_count.append(len(text[i]))

# Convert lists to numpy arrays
word_count = np.array(word_count)
char_count = np.array(char_count)

# Calculate average word lengths
ave_length = np.array(char_count)/np.array(word_count)
#def get_stats(var):
#    print("Min:", np.min(var))
#    print("Max:", np.max(var))
#    print("Mean:", np.mean(var))
#    print("Median", np.median(var))
#    print("1st percentile", np.percentile(var, 1))
#    print("95th percentile", np.percentile(var, 95))
#    print("99th percentile", np.percentile(var, 99))
#    print("99.5th Percentile", np.percentile(var, 99.5))
#    print("99.9th Percentile", np.percentile(var, 99.9))
#print("Word count statistics")
#get_stats(word_count)
# Plot word count distribution
sns.distplot(word_count, kde = False, bins = 70, color = 'blue').set_title("Word Count Distribution")
plt.xlabel('Excerpt Length (Words)')
plt.ylabel('Count')
plt.xlim(0, 200)
plt.savefig("word_count.eps")

In [ ]:
# Plot character count distribution
sns.distplot(char_count, kde = False, bins = 100, color = 'blue').set_title("Character Count Distribution")
plt.xlabel('Excerpt Length (Characters)')
plt.ylabel('Count')
plt.xlim(0, 600)
plt.savefig("char_count.eps")
#print("\nAverage length statistics")
#get_stats(ave_length)


In [ ]:
# Plot average excerpt length distribution
sns.distplot(ave_length, kde = False, bins = 70, color = 'blue').set_title("Average Word Length Distribution")
plt.xlabel('Average Excerpt Length (Characters)')
plt.ylabel('Count')
plt.xlim(0, 15)
plt.savefig("ave_length.eps")

In [ ]:
#Preprocessing
word_outliers = np.where(word_count > 150)
word_outliers = np.where(word_count < 2)
length_outliers = np.where(ave_length > 10)
length_outliers = np.where(ave_length < 3.5)
text_string = ''
for i in range(len(text)):
    text_string += text[i].lower()
char_cnt = Counter(text_string)
accented_chars = ['ï', 'é', 'ñ', 'è', 'ö', 'æ', 'ô', 'â', 'á', 'à', 'ê', 'ë']
accented_text = []
for i in range(len(text)):
    for j in text[i]:
        if j in accented_chars:
            accented_text.append(i)    
accented_text = list(set(accented_text)) 

In [ ]:
# Remove invalid character from text
text = [excerpt.replace('\xa0', '') for excerpt in text]
unusual_text = []

for i in range(len(text)):
    for j in text[i]:
        if j == '\xa0':
            unusual_text.append(i)
        
unusual_text = list(set(unusual_text))
 
print('There are', str(len(unusual_text)), 'texts containing the invalid character.')
new_text = []

for excerpt in text:
    while "  " in excerpt:
        excerpt = excerpt.replace("  "," ")
    new_text.append(excerpt)

text = new_text
ctr = 0
for excerpt in text:
    if "  " in excerpt:
        ctr += 1

print('There are', ctr, 'excerpts containing blocks of white space.')
normed_text = []

for i in range(len(text)):
    new = text[i].lower()
    new = new.translate(str.maketrans('','', string.punctuation))
    new = new.replace('“', '').replace('”', '')
    normed_text.append(new)


In [ ]:
#Training and Testing
from sklearn.model_selection import train_test_split
text_train, text_test, author_train, author_test = train_test_split(normed_text, author, test_size = 0.2, random_state = 5)

In [ ]:
from keras.preprocessing import text
def create_n_grams(excerpt_list, n, vocab_size, seq_size):
    n_gram_list = []
    for excerpt in excerpt_list:
        # Remove spaces
        excerpt = excerpt.replace(" ", "")
        # Extract n-grams
        n_grams = [excerpt[i:i + n] for i in range(len(excerpt) - n + 1)]
        new_string = " ".join(n_grams)
        # One hot encode
        hot = text.one_hot(new_string, round(vocab_size*1.3))
        hot_len = len(hot)
        if hot_len >= seq_size:
            hot = hot[0:seq_size]
        else:
            diff = seq_size - hot_len
            extra = [0]*diff
            hot = hot + extra

        n_gram_list.append(hot)
    
    n_gram_array = np.array(n_gram_list)
    return n_gram_array

In [ ]:
def get_vocab_size(excerpt_list, n, seq_size):
    n_gram_list = []
    for excerpt in excerpt_list:
        excerpt = excerpt.replace(" ", "")
        n_grams = [excerpt[i:i + n] for i in range(len(excerpt) - n + 1)]
        gram_len = len(n_grams)
        if gram_len >= seq_size:
            n_grams = n_grams[0:seq_size]
        else:
            diff = seq_size - gram_len
            extra = [0]*diff
            n_grams = n_grams + extra
        
        n_gram_list.append(n_grams)
    
    # Flatten n-gram list
    n_gram_list = list(np.array(n_gram_list).flat)
    n_gram_cnt = Counter(n_gram_list)
    vocab_size = len(n_gram_cnt)
    return vocab_size
for i in range(1, 4):
    vocab_size = get_vocab_size(text_train, i, 350)
    print('Vocab size for n =', i, 'is:', vocab_size)

In [ ]:
# Create n-gram lists
gram1_train = create_n_grams(text_train, 1, 52, 350)
gram2_train = create_n_grams(text_train, 2, 973, 350)
gram3_train = create_n_grams(text_train, 3, 9529, 350)

gram1_test = create_n_grams(text_test, 1, 52, 350)
gram2_test = create_n_grams(text_test, 2, 973, 350)
gram3_test = create_n_grams(text_test, 3, 9529, 350)

max_1gram = np.max(gram1_train)
max_2gram = np.max(gram2_train)
max_3gram = np.max(gram3_train)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

def process_data(excerpt_list):
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    
    processed = []
    
    for excerpt in excerpt_list:
        new = excerpt.split()
        word_list = [porter.stem(w) for w in new if not w in stop_words]
        word_list = " ".join(word_list)
        processed.append(word_list)
    
    return processed
processed_train = process_data(text_train)
processed_test = process_data(text_test)

In [ ]:
# Bag of words and one hot
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents = 'ascii', stop_words = 'english', min_df = 6)
vectorizer.fit(processed_train)

# Create feature vectors
words_train = vectorizer.transform(processed_train)
words_test = vectorizer.transform(processed_test)
author_lb = preprocessing.LabelBinarizer()

author_lb.fit(author_train)
author_train_hot = author_lb.transform(author_train)
author_test_hot = author_lb.transform(author_test)

In [ ]:
#CNN
def define_model(input_len, output_size, vocab_size, embedding_dim, verbose = True,
                drop_out_pct = 0.25, conv_filters = 500, activation_fn = 'relu', pool_size = 2, learning = 0.0001):
    inputs1 = Input(shape = (input_len,))
    embedding1 = Embedding(vocab_size, embedding_dim)(inputs1)
    drop1 = Dropout(drop_out_pct)(embedding1)
    conv1 = Conv1D(filters = conv_filters, kernel_size = 3, activation = activation_fn)(drop1)
    pool1 = MaxPooling1D(pool_size = pool_size)(conv1)
    flat1 = Flatten()(pool1)

    inputs2 = Input(shape = (input_len,))
    embedding2 = Embedding(vocab_size, embedding_dim)(inputs2)
    drop2 = Dropout(drop_out_pct)(embedding2)
    conv2 = Conv1D(filters = conv_filters, kernel_size = 4, activation = activation_fn)(drop2)
    pool2 = MaxPooling1D(pool_size = pool_size)(conv2)
    flat2 = Flatten()(pool2)

    inputs3 = Input(shape = (input_len,))
    embedding3= Embedding(vocab_size, embedding_dim)(inputs3)
    drop3 = Dropout(drop_out_pct)(embedding3)
    conv3 = Conv1D(filters = conv_filters, kernel_size = 5, activation = activation_fn)(drop3)
    pool3 = MaxPooling1D(pool_size = pool_size)(conv3)
    flat3 = Flatten()(pool3)
    
    merged = concatenate([flat1, flat2, flat3])
    output = Dense(output_size, activation = 'softmax')(merged)
    model = Model(inputs = [inputs1, inputs2, inputs3], outputs = output)
    model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr = learning), metrics=['accuracy'])
    
    if verbose:
        print(model.summary())
        
    return model

In [ ]:
# Create the 1-gram model
gram1_model = define_model(350, 8, max_1gram + 1, 26)

In [ ]:
# Train 1-gram CNN
gram1_model.fit([gram1_train, gram1_train, gram1_train], author_train_hot, epochs=5, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
# Create the 2-gram model
gram2_model = define_model(350, 8, max_2gram + 1, 300)

In [ ]:
# Train 2-gram CNN
gram2_model.fit([gram2_train, gram2_train, gram2_train], author_train_hot, epochs=5, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
# Create the 3-gram model
gram3_model = define_model(350, 8, max_3gram + 1, 600)

In [ ]:
# Train 3-gram CNN
gram3_model.fit([gram3_train, gram3_train, gram3_train], author_train_hot, epochs=5, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
# SVM
svm = SVC()
params = {'kernel': ['linear'], 'C':[1, 10, 100]}
scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(svm, params, scoring = scorer, verbose = 50)

In [ ]:
# Fit bag of words svm
np.random.seed(6)
word_svm = grid_obj.fit(words_train, author_train)
print(word_svm.best_estimator_)
print(word_svm.cv_results_)

In [ ]:
def define_model2(input_len, output_size, vocab_size, embedding_dim, verbose = True,
                drop_out_pct = 0.25, conv_filters = 500, activation_fn = 'relu', pool_size = 2, learning = 0.0001):
    # Channel 1
    inputs1 = Input(shape = (input_len,))
    embedding1 = Embedding(vocab_size, embedding_dim)(inputs1)
    drop1 = Dropout(drop_out_pct)(embedding1)
    conv1 = Conv1D(filters = conv_filters, kernel_size = 3, activation = activation_fn)(drop1)
    pool1 = MaxPooling1D(pool_size = pool_size)(conv1)
    flat1 = Flatten()(pool1)
    
    # Channel 2
    inputs2 = Input(shape = (input_len,))
    embedding2 = Embedding(vocab_size, embedding_dim)(inputs2)
    drop2 = Dropout(drop_out_pct)(embedding2)
    conv2 = Conv1D(filters = conv_filters, kernel_size = 4, activation = activation_fn)(drop2)
    pool2 = MaxPooling1D(pool_size = pool_size)(conv2)
    flat2 = Flatten()(pool2)

    # Channel 3
    inputs3 = Input(shape = (input_len,))
    embedding3= Embedding(vocab_size, embedding_dim)(inputs3)
    drop3 = Dropout(drop_out_pct)(embedding3)
    conv3 = Conv1D(filters = conv_filters, kernel_size = 5, activation = activation_fn)(drop3)
    pool3 = MaxPooling1D(pool_size = pool_size)(conv3)
    flat3 = Flatten()(pool3)
    
    # Channel 4
    inputs4 = Input(shape = (input_len,))
    embedding4 = Embedding(vocab_size, embedding_dim)(inputs4)
    drop4 = Dropout(drop_out_pct)(embedding4)
    conv4 = Conv1D(filters = conv_filters, kernel_size = 6, activation = activation_fn)(drop4)
    pool4 = MaxPooling1D(pool_size = pool_size)(conv4)
    flat4 = Flatten()(pool4)
    merged = concatenate([flat1, flat2, flat3, flat4])
    output = Dense(output_size, activation = 'softmax')(merged)
    model = Model(inputs = [inputs1, inputs2, inputs3, inputs4], outputs = output)
    model.compile(loss='categorical_crossentropy', optimizer = Adam(lr = learning), metrics=['accuracy'])
    if verbose:
        print(model.summary())
    return model
# Create the 3-gram model
gram3_model2 = define_model2(350, 8, max_3gram + 1, 600)


In [ ]:
# Train 3-gram CNN
gram3_model2.fit([gram3_train, gram3_train, gram3_train, gram3_train], author_train_hot, epochs=5, batch_size=32, 
                verbose = 1, validation_split = 0.2)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)  
    df_cm = pd.DataFrame(cm, index = classes,
                  columns = classes)
    sns.heatmap(df_cm, annot=True, cmap = cmap)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title(title)

In [ ]:
#Model 1 (3-gram CNN)

t0 = time.time()
# Fit model
model1 = define_model(350, 8, max_3gram + 1, 600)
model1.fit([gram3_train, gram3_train, gram3_train], author_train_hot, epochs=5, batch_size=32, 
           verbose = 1, validation_split = 0.2)
t1 = time.time()

# Predict values for test set
author_pred1 = model1.predict([gram3_test, gram3_test, gram3_test])
t2 = time.time()
# Reverse one-hot encoding of labels
author_pred1 = author_lb.inverse_transform(author_pred1)
# Evaluate
accuracy = accuracy_score(author_test, author_pred1)
precision, recall, f1, support = score(author_test, author_pred1)
ave_precision = np.average(precision, weights = support/np.sum(support))
ave_recall = np.average(recall, weights = support/np.sum(support))
ave_f1 = np.average(f1, weights = support/np.sum(support))
confusion = confusion_matrix(author_test, author_pred1, labels = ['Alcott', 'Stoker', 'Twain', 'Austen', 
                                                                  'Collins', 'Doyle', 'Montgomery', 'Bronte'])
    
print("Accuracy:", accuracy)
print("Ave. Precision:", ave_precision)
print("Ave. Recall:", ave_recall)
print("Ave. F1 Score:", ave_f1)
print("Training Time:", (t1 - t0), "seconds")
print("Prediction Time:", (t2 - t1), "seconds")
print("Confusion Matrix:\n", confusion)
plot_confusion_matrix(confusion, classes=['Alcott', 'Stoker', 'Twain', 'Austen', 
                                                                  'Collins', 'Doyle', 'Montgomery', 'Bronte'],\
                      normalize=True, title='Normalized Confusion Matrix - Model 1')

plt.savefig("confusion1.eps")

In [ ]:
#Model 2 (Bag of words SVM)
np.random.seed(28)
t0 = time.time()
model2 = SVC(C = 1, kernel = 'linear')
model2.fit(words_train, author_train)

t1 = time.time()
author_pred2 = model2.predict(words_test)

t2 = time.time()
accuracy = accuracy_score(author_test, author_pred2)
precision, recall, f1, support = score(author_test, author_pred2)
ave_precision = np.average(precision, weights = support/np.sum(support))
ave_recall = np.average(recall, weights = support/np.sum(support))
ave_f1 = np.average(f1, weights = support/np.sum(support))
confusion = confusion_matrix(author_test, author_pred2, labels =  ['Alcott', 'Stoker', 'Twain', 'Austen', 
                                                                  'Collins', 'Doyle', 'Montgomery', 'Bronte'])
print("Accuracy:", accuracy)
print("Ave. Precision:", ave_precision)
print("Ave. Recall:", ave_recall)
print("Ave. F1 Score:", ave_f1)
print("Training Time:", (t1 - t0), "seconds")
print("Prediction Time:", (t2 - t1), "seconds")
print("Confusion Matrix:\n", confusion)
plot_confusion_matrix(confusion, classes=['Alcott', 'Stoker', 'Twain', 'Austen', 
                                          'Collins', 'Doyle', 'Montgomery', 'Bronte'], \
                      normalize=True, title='Normalized Confusion Matrix - Model 2')

plt.savefig("confusion2.eps")